In [5]:
import numpy as np
import numpy.random as random
import pandas as pd
from pandas import Series, DataFrame
from datetime import datetime, timedelta
import os
import re

In [6]:
# 基本的に以下を行えば準備完了

# 前処理を行うファイルを./data/xxxxxxxxtestx/InsoleSensor/0_defaultに用意する
# 前処理を行うファイルを./data/xxxxxxxxtestx/Opti-track/0_defaultに用意する
# ここでパスを指定する
BASE_DIR = './data/20250529test5'

## P2P-Insole前処理用

In [19]:
# データクリーナー1
# -方向データの削除
# -データを0.01秒刻みに整形

data_path = BASE_DIR + '/InsoleSensor/0_default'
output_path = os.path.join(data_path, '../1_data_clean1')

# 出力先ディレクトリが存在しない場合は作成
os.makedirs(output_path, exist_ok=True)

csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]

def resample_and_interpolate(data):
    # タイムスタンプ列の名前を明示的に指定
    timestamp_col = 'Timestamp'
    if timestamp_col not in data.columns:
        print(f"エラー: タイムスタンプ列 '{timestamp_col}' がデータに存在しません。")
        return None

    # タイムスタンプをdatetimeオブジェクトに変換し、インデックスに設定
    # 数値（秒）として保存されていると仮定し、基準日（エポック）からの経過時間として処理
    data[timestamp_col] = pd.to_datetime(data[timestamp_col], unit='s', origin='unix')
    data = data.set_index(timestamp_col)

    # 重複したインデックスを処理 (最初の出現を保持)
    data = data[~data.index.duplicated(keep='first')]

    # 0.01秒刻みでリサンプリングし、欠損値を線形補間
    data_resampled = data.resample('10ms').mean().interpolate(method='linear')

    return data_resampled

for csv_file in csv_files:
    # CSVファイルの読み込み(header=1 <=> 一行目の//DN:108,SN:35の行を削除)
    file_path = os.path.join(data_path, csv_file)
    data = pd.read_csv(file_path, header=1)

    # 方位データを削除
    if all(col in data.columns for col in ["Mag_x", "Mag_y", "Mag_z"]):
        data = data.drop(["Mag_x", "Mag_y", "Mag_z"], axis=1)

    # リサンプリング処理の適用
    data_resample = resample_and_interpolate(data.copy()) # 元のデータを変更しないようにコピーを渡す

    if data_resample is not None:
        # 出力ファイルの保存
        output_file_path = os.path.join(output_path, csv_file)
        data_resample.to_csv(output_file_path)

        # 処理内容の確認（例として最初の数行を表示）
        print(f'Processed {csv_file}:')
        print(data_resample.head())
    else:
        print(f'Error processing {csv_file}. スキップします。')

Processed 20250529_135111_left.csv:
                               P1        P2   P3        P4        P5  \
Timestamp                                                              
2025-05-29 04:35:12.710  7.546171  1.172479  0.0  0.311902  0.275650   
2025-05-29 04:35:12.720  7.546171  1.018906  0.0  0.305467  0.267399   
2025-05-29 04:35:12.730  7.052063  1.018906  0.0  0.299120  0.267399   
2025-05-29 04:35:12.740  7.546171  1.018906  0.0  0.292861  0.267399   
2025-05-29 04:35:12.750  7.546171  1.018906  0.0  0.292861  0.228688   

                               P6        P7        P8        P9       P10  \
Timestamp                                                                   
2025-05-29 04:35:12.710  1.822273  3.261982  1.093818  2.041016  4.361952   
2025-05-29 04:35:12.720  1.822273  3.144356  1.950592  2.074006  4.361952   
2025-05-29 04:35:12.730  1.822273  3.144356  1.810055  1.882631  4.361952   
2025-05-29 04:35:12.740  1.718828  3.144356  2.024378  2.041016  4.361952 

In [20]:
# データクリーナー2
# -左右のタイムスタンプを一致させる

# フォルダのパスをここに指定してください
data_path = BASE_DIR + '/InsoleSensor/1_data_clean1' 
output_path = os.path.normpath(os.path.join(data_path, "../2_data_clean2"))
os.makedirs(output_path, exist_ok=True)

def sync_csv_by_timestamp(data_path, output_path):
    all_files = os.listdir(data_path)
    left_files = [f for f in all_files if '_left.csv' in f]

    for left_file in left_files:
        match = re.match(r'(.+)_left\.csv', left_file)
        if match:
            base_name = match.group(1)
            right_file = f"{base_name}_right.csv"

            if right_file in all_files:
                left_path = os.path.join(data_path, left_file)
                right_path = os.path.join(data_path, right_file)

                try:
                    left_df = pd.read_csv(left_path)
                    right_df = pd.read_csv(right_path)

                    if 'Timestamp' not in left_df.columns or 'Timestamp' not in right_df.columns:
                        print(f"'{left_file}' または '{right_file}' に 'Timestamp' カラムが存在しません。")
                        continue

                    left_df.set_index('Timestamp', inplace=True)
                    right_df.set_index('Timestamp', inplace=True)

                    common_timestamps = left_df.index.intersection(right_df.index)

                    if common_timestamps.empty:
                        print(f"'{left_file}' と '{right_file}' に共通のタイムスタンプがありません。")
                        continue

                    synced_left_df = left_df.loc[common_timestamps]
                    synced_right_df = right_df.loc[common_timestamps]

                    synced_left_path = os.path.join(output_path, f"{base_name}_left.csv")
                    synced_right_path = os.path.join(output_path, f"{base_name}_right.csv")

                    synced_left_df.to_csv(synced_left_path)
                    synced_right_df.to_csv(synced_right_path)

                    print(f"'{left_file}' と '{right_file}' を同期し、'{synced_left_path}' と '{synced_right_path}' に保存しました。")

                except Exception as e:
                    print(f"エラー: '{left_file}' と '{right_file}' の処理中にエラーが発生しました: {e}")
    
sync_csv_by_timestamp(data_path, output_path)
print("処理が完了しました。")

'20250529_135111_left.csv' と '20250529_135111_right.csv' を同期し、'data\20250529test5\InsoleSensor\2_data_clean2\20250529_135111_left.csv' と 'data\20250529test5\InsoleSensor\2_data_clean2\20250529_135111_right.csv' に保存しました。
'20250529_143200_left.csv' と '20250529_143200_right.csv' を同期し、'data\20250529test5\InsoleSensor\2_data_clean2\20250529_143200_left.csv' と 'data\20250529test5\InsoleSensor\2_data_clean2\20250529_143200_right.csv' に保存しました。
'20250529_150203_left.csv' と '20250529_150203_right.csv' を同期し、'data\20250529test5\InsoleSensor\2_data_clean2\20250529_150203_left.csv' と 'data\20250529test5\InsoleSensor\2_data_clean2\20250529_150203_right.csv' に保存しました。
'20250529_155853_left.csv' と '20250529_155853_right.csv' を同期し、'data\20250529test5\InsoleSensor\2_data_clean2\20250529_155853_left.csv' と 'data\20250529test5\InsoleSensor\2_data_clean2\20250529_155853_right.csv' に保存しました。
'20250529_165625_left.csv' と '20250529_165625_right.csv' を同期し、'data\20250529test5\InsoleSensor\2_data_clean2\20250529_16

In [21]:
# P2P-insoleのタイムスタンプがUTC(協定世界時)に合わせられている場合はこのコードを実行する

# フォルダのパスをここに指定してください
data_path = BASE_DIR + '/InsoleSensor/2_data_clean2' 
output_path = BASE_DIR + '/InsoleSensor/2_data_clean2' 

csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]

def convert_utc_to_jst(csv_file_path):
    # CSVファイルを読み込む
    df = pd.read_csv(csv_file_path)

    # タイムスタンプ列をdatetime型に変換 (UTCとして認識)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True)

    # UTCからJSTに変換 (+9時間)
    df['Timestamp'] = df['Timestamp'].dt.tz_convert('Asia/Tokyo')

    return df
    

for csv_file in csv_files:
    # CSVファイルの読み込み
    file_path = os.path.join(data_path, csv_file)
    data = pd.read_csv(file_path)

    # UTCからJSTに変換
    converted_df = convert_utc_to_jst(file_path)
   
    # 出力ファイルの保存
    output_file_path = os.path.join(output_path, csv_file)
    converted_df.to_csv(output_file_path, index=False)   


## Opti-Track前処理用

In [22]:
# データクリーナー1
# -データ頭の詳細情報を削除
# -Frame, 偶数ポイント行の削除(ローテーションデータの削除)
# -ボーン以外のデータを削除するコード(X_42以降)

# 0_defaultフォルダに生データを入れる
data_path = BASE_DIR + '/Opti-track/0_default'
output_path = os.path.normpath(os.path.join(data_path, "../1_data_clean1"))
os.makedirs(output_path, exist_ok=True)

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]

# CSVファイルごとに処理を行う
for csv_file in csv_files:
    # CSVファイルを読み込み(エラーが出た場合はheaderを調整)
    file_path = os.path.join(data_path, csv_file)
    data = pd.read_csv(file_path, header = 5)

    display(data.head())

    data = data.drop(['Frame','X','Y','Z'],axis=1)
    for n in range(2, 41, 2):
        data = data.drop(['X.'+ str(n), 'Y.'+ str(n), 'Z.'+ str(n)], axis=1)

    for n in range(42, 600, 1):
        try:
            data = data.drop(['X.'+ str(n), 'Y.'+ str(n), 'Z.'+ str(n)], axis=1)
        except KeyError :
            print(['X.'+ str(n), 'Y.'+ str(n), 'Z.'+ str(n)], "not found in axis")
            break
    
    data.to_csv(output_path + "/" +csv_file, index=False)

,Frame,Time (Seconds),X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,...,Z.79,X.80,Y.80,Z.80,X.81,Y.81,Z.81,X.82,Y.82,Z.82
0,0,0.00,0.246202,-0.629273,1.025148,-255.535690,979.681641,-157.194031,-9.453384,-0.722693,...,-246.450363,-324.504181,49.727360,9.626674,-392.432098,68.207405,42.337955,-438.652710,38.249928,-21.667440
1,1,0.01,1.203027,-0.628509,0.913134,-252.255890,980.437134,-131.647461,-8.621145,-0.377389,...,-246.449005,-324.506439,49.726711,9.628391,-392.429779,68.210510,42.340584,-438.654297,38.251797,-21.664724
2,2,0.02,1.666461,-1.165649,0.909740,-249.937622,979.319275,-114.547943,-10.869576,-0.898690,...,-246.447830,-324.508453,49.726055,9.629638,-392.425995,68.213371,42.343731,-438.655731,38.252956,-21.662756
3,3,0.03,2.020126,-1.595325,1.027767,-249.101959,976.624023,-106.767967,-13.275225,-1.291459,...,-246.446716,-324.509949,49.725750,9.629978,-392.420563,68.215736,42.347130,-438.657318,38.253765,-21.661901
4,4,0.04,2.411037,-1.899013,1.215094,-248.724304,974.052795,-103.890999,-14.996196,-1.508579,...,-246.445251,-324.510956,49.725677,9.629429,-392.417114,68.216812,42.349762,-438.658936,38.254795,-21.661982


['X.83', 'Y.83', 'Z.83'] not found in axis


,Frame,Time (Seconds),X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,...,Z.79,X.80,Y.80,Z.80,X.81,Y.81,Z.81,X.82,Y.82,Z.82
0,0,0.00,-0.379662,-5.608442,0.097640,-192.636795,1014.180176,104.825706,-10.419440,-5.437990,...,41.779690,-270.538818,45.254688,301.185547,-316.063019,82.947998,346.731201,-380.354736,47.390274,301.439117
1,1,0.01,0.637062,-4.845630,0.348952,-194.609818,1011.475342,133.452896,-10.352758,-4.540335,...,41.779362,-270.534180,45.260555,301.187927,-316.067230,82.954498,346.729584,-380.355408,47.390404,301.432556
2,2,0.02,1.249820,-4.715539,-0.002805,-192.457520,1009.671204,152.163116,-13.141925,-4.606335,...,41.778534,-270.531158,45.265675,301.188873,-316.070618,82.959915,346.728180,-380.355225,47.391144,301.426666
3,3,0.03,1.806125,-4.569025,-0.280430,-193.253860,1007.337463,161.848572,-15.974547,-4.567163,...,41.778046,-270.529297,45.270393,301.189240,-316.072418,82.963852,346.727722,-380.354370,47.392025,301.422272
4,4,0.04,2.421489,-4.269368,-0.309544,-195.917999,1005.245667,166.291321,-17.862967,-4.312128,...,41.778286,-270.527649,45.274452,301.189789,-316.072449,82.966133,346.728210,-380.353302,47.392822,301.419250


['X.83', 'Y.83', 'Z.83'] not found in axis


,Frame,Time (Seconds),X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,...,Z.79,X.80,Y.80,Z.80,X.81,Y.81,Z.81,X.82,Y.82,Z.82
0,0,0.00,0.849145,-12.345446,1.712558,-128.486954,985.708008,-573.708374,-7.146493,-12.209144,...,-638.227234,-246.819122,52.190887,-371.637848,-306.494995,89.412270,-355.111664,-347.153229,50.775833,-417.288239
1,1,0.01,1.771228,-9.668205,2.132622,-129.000916,985.425415,-547.236267,-6.551453,-8.309814,...,-638.230042,-246.818863,52.189743,-371.636322,-306.505188,89.412460,-355.100861,-347.155365,50.790096,-417.317932
2,2,0.02,2.204849,-8.427471,1.441038,-117.554596,983.036438,-530.100342,-8.633832,-7.080420,...,-638.231934,-246.818802,52.188023,-371.636139,-306.510803,89.411835,-355.097534,-347.162476,50.796875,-417.329529
3,3,0.03,2.606601,-7.647346,0.725353,-112.968903,978.646240,-521.463501,-10.802356,-6.809130,...,-638.233154,-246.818863,52.186344,-371.636688,-306.513855,89.410507,-355.097565,-347.171478,50.798512,-417.328674
4,4,0.04,3.131000,-6.688735,0.411632,-115.764458,974.680359,-518.128235,-12.233830,-6.346881,...,-638.233643,-246.819077,52.185349,-371.636414,-306.517517,89.409576,-355.095367,-347.177765,50.800491,-417.328796


...

['X.83', 'Y.83', 'Z.83'] not found in axis


,Frame,Time (Seconds),X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,...,Z.79,X.80,Y.80,Z.80,X.81,Y.81,Z.81,X.82,Y.82,Z.82
0,0,0.00,2.434529,-6.008333,-0.032400,-81.421974,912.765198,-414.891602,1.389836,-7.032369,...,-539.010071,-140.193527,53.063663,-290.156647,-212.450882,76.900909,-263.874939,-257.671112,50.778503,-332.070099
1,1,0.01,1.611377,-3.503198,0.029697,-90.223152,910.000244,-423.981476,-3.375305,-3.736902,...,-539.010254,-140.194016,53.041855,-290.141327,-212.442108,76.911369,-263.863831,-257.717773,50.814678,-332.082336
2,2,0.02,1.596994,-1.827886,0.040836,-93.436775,909.148193,-419.395538,-7.624899,-1.827079,...,-539.011719,-140.200241,53.045311,-290.158203,-212.430573,76.905037,-263.877777,-257.679230,50.781544,-332.057556
3,3,0.03,1.893498,-1.173577,-0.007165,-92.900528,908.249573,-413.967712,-11.183733,-1.543559,...,-539.006287,-140.208923,53.063080,-290.157684,-212.449463,76.923111,-263.886993,-257.674469,50.783878,-332.063599
4,4,0.04,2.311216,-1.018833,-0.212231,-89.935776,907.143982,-410.382599,-13.834515,-1.837655,...,-539.029297,-140.217072,53.074406,-290.164734,-212.457809,76.939285,-263.904236,-257.673035,50.787529,-332.075531


['X.83', 'Y.83', 'Z.83'] not found in axis


,Frame,Time (Seconds),X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,...,Z.79,X.80,Y.80,Z.80,X.81,Y.81,Z.81,X.82,Y.82,Z.82
0,0,0.00,1.683662,-3.341145,-1.008406,28.333065,896.042358,-137.622910,-2.486427,-3.514931,...,-266.714600,-31.776361,50.300091,-11.412215,-90.871460,78.411026,28.204958,-145.033340,47.004738,-29.708668
1,1,0.01,1.673481,-2.686046,-0.647714,23.083679,894.470703,-129.121201,-5.022021,-1.855059,...,-266.728882,-31.782921,50.307606,-11.392518,-90.853210,78.409279,28.220579,-145.025330,46.931881,-29.692863
2,2,0.02,1.633515,-3.580282,-0.487096,23.444918,893.234680,-117.600136,-9.506097,-2.729804,...,-266.710510,-31.777710,50.339474,-11.390921,-90.858803,78.414238,28.225122,-145.038864,46.964493,-29.670862
3,3,0.03,1.650266,-4.789030,-0.225147,23.732502,891.582397,-110.007477,-13.594760,-4.675923,...,-266.721100,-31.780680,50.302898,-11.407642,-90.850266,78.413818,28.216854,-145.029312,46.964535,-29.681684
4,4,0.04,1.854750,-5.393074,-0.010137,25.312826,889.739380,-105.503708,-16.168329,-5.841189,...,-266.721832,-31.762878,50.299740,-11.413798,-90.841774,78.400269,28.223886,-145.028442,46.959839,-29.670910


['X.83', 'Y.83', 'Z.83'] not found in axis


In [23]:
# データクリーナー2
# タイムスタンプを作成するコード
data_path = BASE_DIR + '/Opti-track/1_data_clean1'
output_path = os.path.normpath(os.path.join(data_path, "../2_data_clean2"))
os.makedirs(output_path, exist_ok=True)

def convert_time_seconds_to_timestamp(filename):
    try:
        # CSVファイルを読み込む
        df = pd.read_csv(filename)
    except FileNotFoundError:
        print(f"エラー: ファイル '{filename}' が見つかりません。")
        return None
    except Exception as e:
        print(f"エラー: CSVファイルの読み込み中にエラーが発生しました: {e}")
        return None

    try:
        # 正規表現でファイル名から日時とAM/PMを抽出
        base_filename = os.path.basename(filename)
        match = re.search(r'(\d{4}-\d{2}-\d{2})\s+(\d{2})\.(\d{2})\.(\d{2})\s+(AM|PM)', base_filename)
        if match:
            date_part = match.group(1).replace('-', '/')
            hour_part = int(match.group(2))
            minute_part = int(match.group(3))
            second_part = int(match.group(4))
            am_pm_part = match.group(5)
            start_datetime = datetime(int(date_part.split('/')[0]),
                                      int(date_part.split('/')[1]),
                                      int(date_part.split('/')[2]),
                                      hour_part, minute_part, second_part)
            if am_pm_part == 'PM' and hour_part != 12:
                start_datetime = start_datetime + timedelta(hours=12)
            elif am_pm_part == 'AM' and hour_part == 12:
                start_datetime = start_datetime - timedelta(hours=12) # midnight case

        else:
            print(f"エラー: ファイル '{base_filename}' から日時を抽出できませんでした。ファイル名が予期された形式ではありません。")
            return df

    except ValueError as e:
        print(f"エラー: ファイル '{base_filename}' から抽出した日時を解析できませんでした: {e}")
        return df  # ファイルは読み込めたので、変換前のDataFrameを返す

    # Time (Second) 列をtimedeltaオブジェクトに変換
    try:
        time_deltas = pd.to_timedelta(df['Time (Seconds)'], unit='s')
    except KeyError:
        print(f"エラー: ファイル '{base_filename}' に 'Time (Seconds)' という列が見つかりません。")
        return df  # 'Time (Second)' 列がない場合は、元のDataFrameを返す

    # 開始日時にtimedeltaを加算してTimestamp列を生成
    timestamp_column = start_datetime + time_deltas

    # データフレームにTimestamp列を追加
    df['Time (Seconds)'] = timestamp_column
    new_df = df.rename(columns={'Time (Seconds)' : 'Timestamp'})

    # タイムスタンプ列をdatetime型に変換
    new_df['Timestamp'] = pd.to_datetime(new_df['Timestamp'], utc=True)

    return new_df

def process_all_csv_in_folder(folder_path):
    if not os.path.isdir(folder_path):
        print(f"エラー: 指定されたパス '{folder_path}' はフォルダではありません。")
        return

    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            full_filepath = os.path.join(folder_path, filename)
            print(f"処理中のファイル: {full_filepath}")
            df_with_timestamp = convert_time_seconds_to_timestamp(full_filepath)
            if df_with_timestamp is not None:
                try:
                    df_with_timestamp.to_csv(os.path.join(output_path, filename), index=False)
                    print(f"処理後のファイルは '{output_path}' に保存されました。")
                except Exception as e:
                    print(f"エラー: 処理後のファイルの保存中にエラーが発生しました: {e}")
            print("-" * 30)


# フォルダ内のすべてのCSVファイルを処理
process_all_csv_in_folder(data_path)
print("処理が完了しました。")

処理中のファイル: ./data/20250529test5/Opti-track/1_data_clean1\Take 2025-05-29 01.35.10 PM.csv
処理後のファイルは 'data\20250529test5\Opti-track\2_data_clean2' に保存されました。
------------------------------
処理中のファイル: ./data/20250529test5/Opti-track/1_data_clean1\Take 2025-05-29 02.15.59 PM.csv
処理後のファイルは 'data\20250529test5\Opti-track\2_data_clean2' に保存されました。
------------------------------
処理中のファイル: ./data/20250529test5/Opti-track/1_data_clean1\Take 2025-05-29 02.46.00 PM.csv
処理後のファイルは 'data\20250529test5\Opti-track\2_data_clean2' に保存されました。
------------------------------
処理中のファイル: ./data/20250529test5/Opti-track/1_data_clean1\Take 2025-05-29 03.42.51 PM.csv
処理後のファイルは 'data\20250529test5\Opti-track\2_data_clean2' に保存されました。
------------------------------
処理中のファイル: ./data/20250529test5/Opti-track/1_data_clean1\Take 2025-05-29 04.39.15 PM.csv
処理後のファイルは 'data\20250529test5\Opti-track\2_data_clean2' に保存されました。
------------------------------
処理中のファイル: ./data/20250529test5/Opti-track/1_data_clean1\Take 2025-05-29 04.

In [ ]:
# 位置固定処理
# 中心点は股関節（root）位置の列 ['X.1', 'Z.1'](高さ以外に処理を施す)
data_path = BASE_DIR + '/Opti-track/2_data_clean2'
output_path = os.path.normpath(os.path.join(data_path, "../3_PositionHeld"))
os.makedirs(output_path, exist_ok=True)

# 位置固定処理関数
def position_held(file_path):
    
    df = pd.read_csv(file_path)

    # 3. 各軸ごとに並進ベクトルを差分
    for axis in ['X', 'Z']:
        root_col = f'{axis}.1'
        # 同じ接頭辞をもつ全ての列を取得
        target_cols = [c for c in df.columns if c.startswith(f'{axis}.')]
        # 各列から root_col を引く
        df[target_cols] = df[target_cols].sub(df[root_col], axis=0)

    df.to_csv(output_path + "/" +csv_file, index=False)
    return

# 実行
csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]
for csv_file in csv_files:
    file_path = os.path.join(data_path, csv_file)
    position_held(file_path)


print("処理が完了しました。")

処理が完了しました。


In [ ]:

# まだうまく動かないため使用不可

# # 正面固定処理
# data_path = BASE_DIR + '/Opti-track/3_PositionHeld'
# output_path = os.path.normpath(os.path.join(data_path, "../4_RotationHeld"))
# os.makedirs(output_path, exist_ok=True)

# # --- 基準ジョイント列名の定義 ---
# left_hip_cols   = ['X.35',  'Y.35',  'Z.35']   # 左腰関節
# right_hip_cols  = ['X.27',  'Y.27',  'Z.27']   # 右腰関節
# spine_cols      = ['X.5',   'Y.5',   'Z.5']    # 胸部または背骨中点

# # 正面固定処理関数
# def Rotation_held(file_path, left_hip_cols, right_hip_cols, spine_cols):
#     df = pd.read_csv(file_path)

#     # 関節座標の列リスト（Timestamp以外の全て）
#     joint_cols = [c for c in df.columns if c != 'Timestamp']

#     # フレームごとに回転行列を作り、全関節に適用
#     rotated_rows = []
#     for _, row in df.iterrows():
#         # 3.1 キーとなるポイントを取得
#         L = row[left_hip_cols].values.astype(float)
#         R = row[right_hip_cols].values.astype(float)
#         S = row[spine_cols].values.astype(float)
        
#         # 骨盤座標系の単位ベクトルを計算
#         # X軸（左右ベクトル）：右→左
#         x_axis = (L - R)
#         x_axis /= np.linalg.norm(x_axis)
#         # Z軸（前方ベクトル）：腰中点→胸部
#         pelvis_mid = (L + R) / 2.0
#         y_axis = (S - pelvis_mid)
#         y_axis /= np.linalg.norm(y_axis)
#         # Y軸（上向きベクトル）：X軸×Z軸 の外積（右手系）
#         z_axis = np.cross(x_axis, y_axis)
#         z_axis /= np.linalg.norm(z_axis)
#         # 再直交化：Z軸＝Y×X
#         y_axis = np.cross(z_axis, x_axis)
#         y_axis /= np.linalg.norm(y_axis)
        
#         # 回転行列 R を組み立て（ワールド←骨盤局所）
#         R_mat = np.stack([x_axis, y_axis, z_axis], axis=1)  # 3×3
        
#         # すべての関節座標に逆回転 R^T を適用
#         coords = row[joint_cols].values.reshape(-1, 3)      # N_joints×3
#         rotated = coords.dot(R_mat.T)                      # N_joints×3
        
#         rotated_rows.append(rotated.flatten())

#     rotated_df = pd.DataFrame(rotated_rows, columns=joint_cols)
#     rotated_df['Timestamp'] = df['Timestamp']
#     rotated_df.to_csv(output_path + "/" +csv_file, index=False)
        
#     return

# # 実行
# csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]
# for csv_file in csv_files:
#     file_path = os.path.join(data_path, csv_file)
#     Rotation_held(file_path, left_hip_cols, right_hip_cols, spine_cols)

# print("処理が完了しました。")

In [ ]:
# 無駄な骨格ポイントを削減するコード + 列名を整えるコード(列名をX.1, Y.1, Z.1, X.2, Y.2, ...のように1から17点を使用した列名に変換する)
data_path = BASE_DIR + '/Opti-track/3_PositionHeld'
output_path = os.path.normpath(os.path.join(data_path, "../4_good_skeleton"))
os.makedirs(output_path, exist_ok=True)

# 不要な骨格ポイント削除
def make_good_skeleton(file_path, csv_file, output_path):
    df = pd.read_csv(file_path)

    # ポイントを指定してドロップする処理を書く(除くポイントは 3, 5, 11, 19)
    # この処理を行うとこれまでの骨格定義が使用できなくなることに注意
    # データのフォーマットを整える必要あり、(x.1, y.1, z.1, x.3, y.3, z.3, ,,,,,)(←もうすでにおかしい)
    # モデルの次元、可視化用コードのデータロード、骨格定義は変更しなくてはならない

    df = row_regulationer(df)
    df = df.drop(["X.3", "Y.3", "Z.3", "X.5", "Y.5", "Z.5","X.11", "Y.11", "Z.11","X.19", "Y.19", "Z.19", ], axis=1)
    df = row_regulationer(df)

    df.to_csv(output_path + "/" +csv_file, index=False)

    return

def row_regulationer(df):
    tmp_columns = []
    point_i = 1

    # tmp_columns.append('Timestamp')
    for i in range(1, len(df.columns), 3):
        tmp_columns.append(f"X.{point_i}")
        tmp_columns.append(f"Y.{point_i}")
        tmp_columns.append(f"Z.{point_i}")
        point_i = point_i + 1

    df.columns = tmp_columns

    return df


# 実行
csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]
for csv_file in csv_files:
    file_path = os.path.join(data_path, csv_file)
    make_good_skeleton(file_path, csv_file, output_path)


print("処理が完了しました。")

処理が完了しました。


## P2P-insole, Optitrack共通処理

- ファイル名をわかりやすいものに変換する

P2P-Insole<br>
ex) T001S001_Insole_l.csv<br>
ex) T001S001_Insole_r.csv<br>
<br>

Opti-track<br>
ex) T001S001_skeleton.csv
<br>

T: test
S: subject


- P2P-insoleとOptitrackのタイムスタンプを同期する(手作業)(今後この作業も自動化する)

In [ ]:
# # タイムスタンプを同期するコード
# # 完成したら下のタイムスタンプを削除するコードと合わせる
# data_path_P2PInsole = BASE_DIR + '/InsoleSensor/2-5_renamed'
# output_path_P2PInsole = os.path.normpath(os.path.join(data_path_P2PInsole, "../3_timestamp_syncro"))
# os.makedirs(output_path_P2PInsole, exist_ok=True)

# data_path_optitrack = BASE_DIR + '/Opti-track/4-5_renamed'
# output_path_optitrack = os.path.normpath(os.path.join(data_path_optitrack, "../5_timestamp_syncro"))
# os.makedirs(output_path_optitrack, exist_ok=True)

# # タイムスタンプの同期
# def timestamp_syncro(insole_file, skeleton_file, output_path_p2PInsole, output_path_optitrack):
    
#     return

# # 実行
# # 各フォルダ内のファイル名を参照してタグが同じものどうしをタイムスタンプ同期する
# csv_files = [f for f in os.listdir(data_path_optitrack) if f.endswith('.csv')]
# for csv_file in csv_files:
#     file_path = os.path.join(data_path_optitrack, csv_file)
#     timestamp_syncro(insole_file, skeleton_file, output_path_insole, output_path_optitrack)

# print("処理が完了しました。")

In [ ]:
# タイムスタンプ削除
# /y_finalに保存する

data_path_P2PInsole = BASE_DIR + '/InsoleSensor/3_timestamp_syncro'
output_path_P2PInsole = os.path.normpath(os.path.join(data_path_P2PInsole, "../4_rem_timestamp"))
os.makedirs(output_path_P2PInsole, exist_ok=True)

data_path_optitrack = BASE_DIR + '/Opti-track/5_timestamp_syncro'
output_path_optitrack = os.path.normpath(os.path.join(data_path_optitrack, "../6_rem_timestamp"))
os.makedirs(output_path_optitrack, exist_ok=True)

# タイムスタンプ削除
def remove_timestamp(file_path, csv_file, output_path):
    df = pd.read_csv(file_path)
    df = df.drop('Timestamp', axis=1)
    df.to_csv(output_path + "/" +csv_file, index=False)
    return

# 実行(P2P-Insole)
csv_files = [f for f in os.listdir(data_path_P2PInsole) if f.endswith('.csv')]
for csv_file in csv_files:
    file_path = os.path.join(data_path_P2PInsole, csv_file)
    remove_timestamp(file_path, csv_file, output_path_P2PInsole)

# 実行(opti-track)
csv_files = [f for f in os.listdir(data_path_optitrack) if f.endswith('.csv')]
for csv_file in csv_files:
    file_path = os.path.join(data_path_optitrack, csv_file)
    remove_timestamp(file_path, csv_file, output_path_optitrack)

print("処理が完了しました。")

処理が完了しました。


In [ ]:
# ダウンサンプリングするコード

In [ ]:
# モデルに組み込んでいる前処理をここで行うべきかもしれない
# nan値を変換
# スケーリング(標準化、正規化)
# ガウスフィルター


この処理が完了したらy_finalにデータを移す
